In [ ]:
# ls -l| tail -10

In [ ]:
# #G4
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# cp fingerspelling5.tar.bz2 /media/datastorage/Phong/fingerspelling5.tar.bz2

In [ ]:
# rm fingerspelling5.tar.bz2

In [1]:
cd /media/datastorage/Phong/

/media/datastorage/Phong


In [ ]:
!tar xjf fingerspelling5.tar.bz2

In [ ]:
cd dataset5

In [ ]:
ls -l

In [ ]:
mkdir surrey/C

In [ ]:
mv dataset5/* surrey/C/

In [ ]:
cd ..

In [ ]:
#remove depth files
import glob
import os
import shutil

# get parts of image's path
def get_image_parts(image_path):
    """Given a full path to an image, return its parts."""
    parts = image_path.split(os.path.sep)
    #print(parts)
    filename = parts[2]
    filename_no_ext = filename.split('.')[0]
    classname = parts[1]
    train_or_test = parts[0]

    return train_or_test, classname, filename_no_ext, filename


#del_folders = ['A','B','C','D','E']  
move_folders_1 = ['A','B','D','E']  
move_folders_2 = ['C']

# look for all images in sub-folders
for folder in move_folders_1:
    class_folders = glob.glob(os.path.join(folder, '*'))
        
    for iid_class in class_folders:
        #move depth files
        class_files = glob.glob(os.path.join(iid_class, 'depth*.png'))
        
        print('copying %d files' %(len(class_files)))
        for idx in range(len(class_files)):        
            src = class_files[idx]
            if "0001" not in src: 
                train_or_test, classname, _, filename = get_image_parts(src)

                dst = os.path.join('train_depth', classname, train_or_test+'_'+ filename)

                # image directory
                img_directory = os.path.join('train_depth', classname)

                # create folder if not existed
                if not os.path.exists(img_directory):
                    os.makedirs(img_directory)

                #copying
                shutil.copy(src, dst)
            else:
                print('ignor: %s' %src)
            
        #move color files    
    for iid_class in class_folders:
        #move depth files
        class_files = glob.glob(os.path.join(iid_class, 'color*.png'))
        
        print('copying %d files' %(len(class_files)))
        for idx in range(len(class_files)):        
            src = class_files[idx]
            train_or_test, classname, _, filename = get_image_parts(src)

            dst = os.path.join('train_color', classname, train_or_test+'_'+ filename)
            
            # image directory
            img_directory = os.path.join('train_color', classname)

            # create folder if not existed
            if not os.path.exists(img_directory):
                os.makedirs(img_directory)
            
            #copying
            shutil.copy(src, dst)
            
# look for all images in sub-folders
for folder in move_folders_2:
    class_folders = glob.glob(os.path.join(folder, '*'))
        
    for iid_class in class_folders:
        #move depth files
        class_files = glob.glob(os.path.join(iid_class, 'depth*.png'))
        
        print('copying %d files' %(len(class_files)))
        for idx in range(len(class_files)):        
            src = class_files[idx]
            if "0001" not in src: 
                train_or_test, classname, _, filename = get_image_parts(src)

                dst = os.path.join('test_depth', classname, train_or_test+'_'+ filename)

                # image directory
                img_directory = os.path.join('test_depth', classname)

                # create folder if not existed
                if not os.path.exists(img_directory):
                    os.makedirs(img_directory)

                #copying
                shutil.copy(src, dst)
            else:
                print('ignor: %s' %src)
            
        #move color files    
    for iid_class in class_folders:
        #move depth files
        class_files = glob.glob(os.path.join(iid_class, 'color*.png'))
        
        print('copying %d files' %(len(class_files)))
        for idx in range(len(class_files)):        
            src = class_files[idx]
            train_or_test, classname, _, filename = get_image_parts(src)

            dst = os.path.join('test_color', classname, train_or_test+'_'+ filename)
            
            # image directory
            img_directory = os.path.join('test_color', classname)

            # create folder if not existed
            if not os.path.exists(img_directory):
                os.makedirs(img_directory)
            
            #copying
            shutil.copy(src, dst)                

In [ ]:
# #/content
%cd ..

In [ ]:
ls -l

In [ ]:
mkdir surrey/C/checkpoints

In [ ]:
cd surrey/

In [2]:
#MUL 1 - Xception - ST

# from keras.applications import MobileNet
from keras.applications import InceptionV3
# from keras.applications import Xception
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from tensorflow.keras.applications import EfficientNetB0

from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, SimpleRNN, LSTM, Flatten, GRU, Reshape

from keras.applications.inception_v3 import preprocess_input
# from tensorflow.keras.applications.efficientnet import preprocess_input
# from keras.applications.mobilenet import preprocess_input
# from keras.applications.xception import preprocess_input

from keras.layers import GaussianNoise

def get_adv_model():
#     f1_base = EfficientNetB0(include_top=False, weights='imagenet', 
#                     input_shape=(299, 299, 3), 
#                     pooling='avg')
#     f1_x = f1_base.output


    f1_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(299,299,3))  
    f1_x = f1_base.output
    f1_x = GlobalAveragePooling2D()(f1_x)   

# f1_x = f1_base.layers[-151].output   #layer 5

# f1_x = GlobalAveragePooling2D()(f1_x)
# f1_x = Flatten()(f1_x)

# f1_x = Reshape([1,1280])(f1_x)  
# f1_x = SimpleRNN(2048, 
#             return_sequences=False,                       
# #             dropout=0.8                                     
#             input_shape=[1,1280])(f1_x)
   
    #Regularization with noise
    f1_x = GaussianNoise(0.1)(f1_x)

    f1_x = Dense(1024, activation='relu')(f1_x)
    f1_x = Dense(24, activation='softmax')(f1_x)
    model_1 = Model(inputs=[f1_base.input],outputs=[f1_x])
    model_1.summary()
    
    return model_1

In [3]:
from keras.callbacks import Callback
import pickle
import sys

#Stop training on val_acc
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

#Save large model using pickle formate instead of h5            
class SaveCheckPoint(Callback):
    def __init__(self, model, dest_folder):
        super(Callback, self).__init__()
        self.model = model
        self.dest_folder = dest_folder
        
        #initiate
        self.best_val_acc = 0
        self.best_val_loss = sys.maxsize #get max value
          
    def on_epoch_end(self, epoch, logs={}):
        val_acc = logs['val_acc']
        val_loss = logs['val_loss']

        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            
            # Save weights in pickle format instead of h5
            print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
            weigh= self.model.get_weights()

            #now, use pickle to save your model weights, instead of .h5
            #for heavy model architectures, .h5 file is unsupported.
            fpkl= open(self.dest_folder, 'wb') #Python 3
            pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
            fpkl.close()
            
#             model.save('tmp.h5')
        elif val_acc == self.best_val_acc:
            if val_loss < self.best_val_loss:
                self.best_val_loss=val_loss
                
                # Save weights in pickle format instead of h5
                print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
                weigh= self.model.get_weights()

                #now, use pickle to save your model weights, instead of .h5
                #for heavy model architectures, .h5 file is unsupported.
                fpkl= open(self.dest_folder, 'wb') #Python 3
                pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
                fpkl.close()                

In [4]:
# Training
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam

import time, os
from math import ceil

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
#     horizontal_flip=True,
#     vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    # preprocessing_function=get_cutout_v2(),
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

NUM_GPU = 1
batch_size = 32

train_set = train_datagen.flow_from_directory('surrey/D/train_color/',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('surrey/D/test_color/',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'st'
# Helper: Save the model.
savedfilename = os.path.join('surrey', 'D', 'checkpoints', 'Surrey_InceptionV3_D_tmp.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_accuracy', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('svhn_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('svhn_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

earlystopping = EarlyStoppingByAccVal(monitor='val_accuracy', value=0.9900, verbose=1)

epochs = 40##!!!
lr = 1e-3
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

# train on multiple-gpus

# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print("Number of GPUs: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model_mul = get_adv_model()
    model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

# result = model_mul.fit_generator(
#     generator = train_set, 
#     steps_per_epoch = step_size_train,
#     validation_data = valid_set,
#     validation_steps = step_size_valid,
#     shuffle=True,
#     epochs=epochs,
#     callbacks=[checkpointer],
# #     callbacks=[csv_logger, checkpointer, earlystopping],
# #     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
#     verbose=1) 

Found 52620 images belonging to 24 classes.
Found 13154 images belonging to 24 classes.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of GPUs: 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/

In [6]:
# Training
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam

import time, os
from math import ceil

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
#     horizontal_flip=True,
#     vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    # preprocessing_function=get_cutout_v2(),
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

NUM_GPU = 1
batch_size = 32

train_set = train_datagen.flow_from_directory('surrey/D/train_color/',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('surrey/D/test_color/',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'st'
# Helper: Save the model.
savedfilename = os.path.join('surrey', 'D', 'checkpoints', 'Surrey_InceptionV3_D.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_accuracy', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('svhn_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('svhn_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

earlystopping = EarlyStoppingByAccVal(monitor='val_accuracy', value=0.9900, verbose=1)

epochs = 40##!!!
lr = 1e-3
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

# train on multiple-gpus

# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print("Number of GPUs: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model_mul = get_adv_model()
    model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[checkpointer],
#     callbacks=[csv_logger, checkpointer, earlystopping],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

Found 52620 images belonging to 24 classes.
Found 13154 images belonging to 24 classes.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of GPUs: 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/

Epoch 1/40
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
1645/1645 [==============================] - ETA: 0s - loss: 1.1503 - accuracy: 0.6177
Epoch 00001: val_accuracy improved from -inf to 0.65402, saving model to surrey/D/checkpoints/Surrey_InceptionV3_D.hdf5
1645/1645 [==============================] - 991s 603ms/step - loss: 1.1503 - accuracy: 0.6177 - val_loss: 1.3164 - val_accuracy: 0.6540
Epoch 2/40
1645/1645 [==============================] - ETA: 0s - loss: 0.2618 - accuracy: 0.9193
Epoch 00002: val_accuracy improved from 0.65402 to 0.68892, saving model to surrey/D/checkpoints/Surrey_InceptionV3_D.hdf5
1645/1645 [==============================] - 994s 604ms/step - loss: 0.2618 - accuracy: 0.9193 - val_loss: 1.2401 - val_accuracy: 0.6889
Epoch 3/40
1645/1645 [==============================] - ETA: 0s - loss: 0.1660 - accuracy: 0.9489
Epoch 00003: val_accuracy did not improve from 0.68892
1645/1645 [==============================] - 993s 60

Epoch 27/40
1645/1645 [==============================] - ETA: 0s - loss: 0.0181 - accuracy: 0.9943
Epoch 00027: val_accuracy did not improve from 0.84203
1645/1645 [==============================] - 917s 557ms/step - loss: 0.0181 - accuracy: 0.9943 - val_loss: 0.8843 - val_accuracy: 0.7789
Epoch 28/40
1645/1645 [==============================] - ETA: 0s - loss: 0.0204 - accuracy: 0.9936
Epoch 00028: val_accuracy did not improve from 0.84203
1645/1645 [==============================] - 918s 558ms/step - loss: 0.0204 - accuracy: 0.9936 - val_loss: 0.9729 - val_accuracy: 0.7875
Epoch 29/40
1645/1645 [==============================] - ETA: 0s - loss: 0.0181 - accuracy: 0.9945
Epoch 00029: val_accuracy did not improve from 0.84203
1645/1645 [==============================] - 918s 558ms/step - loss: 0.0181 - accuracy: 0.9945 - val_loss: 1.0949 - val_accuracy: 0.7866
Epoch 30/40
1645/1645 [==============================] - ETA: 0s - loss: 0.0152 - accuracy: 0.9951
Epoch 00030: val_accuracy im

In [ ]:
ls -l

In [5]:
# Open a strategy scope.
with strategy.scope():
    model_mul.load_weights(os.path.join('surrey', 'D', 'checkpoints', 'Surrey_InceptionV3_D.hdf5'))

In [6]:
model_mul.evaluate(valid_set)

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
412/412 [==============================] - 32s 78ms/step - loss: 0.6947 - accuracy: 0.8572


[0.6946942806243896, 0.8571537137031555]

In [6]:

# Helper: Save the model.
savedfilename = os.path.join('surrey', 'D', 'checkpoints', 'Surrey_InceptionV3_D_L2.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_accuracy', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

epochs = 15##!!!
lr = 1e-4
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

# Open a strategy scope.
with strategy.scope():
    model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[checkpointer],
#     callbacks=[csv_logger, checkpointer, earlystopping],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
1645/1645 [==============================] - ETA: 0s - loss: 0.0083 - accuracy: 0.9976
Epoch 00001: val_accuracy improved from -inf to 0.85062, saving model to surrey/D/checkpoints/Surrey_InceptionV3_D_L2.hdf5
1645/1645 [==============================] - 988s 601ms/step - loss: 0.0083 - accuracy: 0.9976 - val_loss: 0.7266 - val_accuracy: 0.8506
Epoch 2/15
1645/1645 [==============================] - ETA: 0s - loss: 0.0074 - accuracy: 0.9979
Epoch 00002: val_accuracy did not improve from 0.85062
1645/1645 [==============================] - 990s 602ms/step - loss: 0.0074 - accuracy: 0.9979 - val_loss: 0.7056 - val_accuracy: 0.8502
Epoch 3/15
1645/1645 [==============================] - ETA: 0s - loss: 0.0064 - accuracy: 0.9980
Epoch 00003: val_accuracy did not improve from 0.85062
1645/1645 [==============================

In [5]:
# Open a strategy scope.
with strategy.scope():
    model_mul.load_weights(os.path.join('surrey', 'D', 'checkpoints', 'Surrey_InceptionV3_D_L2.hdf5'))

In [6]:
model_mul.evaluate(valid_set)

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
412/412 [==============================] - 32s 78ms/step - loss: 0.7266 - accuracy: 0.8506


[0.7266056537628174, 0.8506157994270325]

In [6]:
# Helper: Save the model.
savedfilename = os.path.join('surrey', 'D', 'checkpoints', 'Surrey_InceptionV3_D_L3.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_accuracy', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

epochs = 15##!!!
lr = 1e-5
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

# Open a strategy scope.
with strategy.scope():
    model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[checkpointer],
#     callbacks=[csv_logger, checkpointer, earlystopping],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
1645/1645 [==============================] - ETA: 0s - loss: 0.0117 - accuracy: 0.9964
Epoch 00001: val_accuracy improved from -inf to 0.87456, saving model to surrey/D/checkpoints/Surrey_InceptionV3_D_L3.hdf5
1645/1645 [==============================] - 992s 603ms/step - loss: 0.0117 - accuracy: 0.9964 - val_loss: 0.5740 - val_accuracy: 0.8746
Epoch 2/15
1645/1645 [==============================] - ETA: 0s - loss: 0.0084 - accuracy: 0.9974
Epoch 00002: val_accuracy improved from 0.87456 to 0.87874, saving model to surrey/D/checkpoints/Surrey_InceptionV3_D_L3.hdf5
1645/1645 [==============================] - 988s 601ms/step - loss: 0.0084 - accuracy: 0.9974 - val_loss: 0.5534 - val_accuracy: 0.8787
Epoch 3/15
1645/1645 [==============================] - ETA: 0s - loss: 0.0067 - accuracy: 0.9979
Epoch 00003: val_accuracy

In [5]:
# Open a strategy scope.
with strategy.scope():
    model_mul.load_weights(os.path.join('surrey', 'D', 'checkpoints', 'Surrey_InceptionV3_D_L3.hdf5'))

In [6]:
model_mul.evaluate(valid_set)

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
412/412 [==============================] - 32s 78ms/step - loss: 0.5114 - accuracy: 0.8879


[0.5114102363586426, 0.8879428505897522]

In [7]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import time, os
from math import ceil

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
#     horizontal_flip=True,
#     vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 32

train_set = train_datagen.flow_from_directory('surrey/D/train_color/',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('surrey/D/test_color/',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

# if NUM_GPU != 1:
predict1=model_mul.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('Surrey_InceptionV3_D_L2_0902.csv')
results.head()

Found 52620 images belonging to 24 classes.
Found 13154 images belonging to 24 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
412/412 [==============================] - 31s 75ms/step


,file_name,predicted1
0,a/D_color_0_0002.png,a
1,a/D_color_0_0003.png,a
2,a/D_color_0_0004.png,a
3,a/D_color_0_0005.png,a
4,a/D_color_0_0006.png,a


In [8]:
cp Surrey_InceptionV3_D_L3_0902.csv /home/bribeiro/Phong/Nat19/Surrey_InceptionV3_D_L3_0902.csv

In [10]:
mkdir surrey/D/npy

In [8]:
np.save(os.path.join('surrey', 'D', 'npy', 'Surrey_InceptionV3_D_L1_0902.npy'), predict1)

In [8]:
np.save(os.path.join('surrey', 'D', 'npy', 'Surrey_InceptionV3_D_L2_0902.npy'), predict1)

In [10]:
np.save(os.path.join('surrey', 'D', 'npy', 'Surrey_InceptionV3_D_L3_0902.npy'), predict1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

test_datagen = ImageDataGenerator(
    rescale = 1./255)

testing_set = test_datagen.flow_from_directory('dataset5/test_color/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 seed=7,
                                                 shuffle=False
#                                                  subset="validation"
                                             )

y_pred = model.predict_generator(testing_set,steps = testing_set.n//testing_set.batch_size)
y_pred = np.argmax(y_pred, axis=1)

y_true = testing_set.classes

print(confusion_matrix(y_true, y_pred))

# print(model.evaluate_generator(testing_set,
#                                steps = testing_set.n//testing_set.batch_size))